# Foxglove Data Platform Demo Notebook

This notebook demonstrates retrieving data and events from Foxglove Data Platform.


In [1]:
# First we create an instance of our client.

from foxglove.data_platform.client import Client, OutputFormat

token = "fox_sk_SBK5ew4HwygFEw9gTdttnLsh3pCtXEoe"
client = Client(token=token)

In [2]:
# Next let's use the client to list some events.

import tabulate

events = client.list_events(device_name="Robot", limit=5, key="position", value="start")
desired_fields = ["id", "metadata", "timestamp"]
filtered_events = [{k: v for k, v in e.items() if k in desired_fields} for e in events]
print(tabulate.tabulate(filtered_events, headers="keys", tablefmt="pretty"))

+----------------------+-----------------------+---------------------------+
|          id          |       metadata        |         timestamp         |
+----------------------+-----------------------+---------------------------+
| evt_Bcf8Q6qI7cOaveRM | {'position': 'start'} | 2018-07-24 03:28:47+00:00 |
+----------------------+-----------------------+---------------------------+


In [3]:
# Now we can download some data from data platform.

import sys
from datetime import datetime, timedelta

start_event = events[-1]
start_time = start_event["timestamp"]
end_time = start_time + timedelta(seconds=1)
device_id = start_event["device_id"]

download_stream = client.download_data(
    device_id=device_id,
    start=start_time,
    end=end_time,
    output_format=OutputFormat.mcap0,
)

data = bytes()
for chunk in download_stream.iter_content(chunk_size=64 * 1024):
    sys.stdout.write(".")
    data += chunk
print()
print(f"read {len(data)} bytes")

...................................................................................
read 3296795 bytes


In [4]:
# Let's unpack some schemas, channels, and messages, from our mcap stream.

from io import BytesIO

from genpy import dynamic
from mcap.mcap0.records import Channel, Message, Schema
from mcap.mcap0.stream_reader import StreamReader

reader = StreamReader(BytesIO(data))
channels = {}
schemas = {}
messages = []
msg_types = {}
for record in reader.records:
    if isinstance(record, Schema):
        schemas[record.id] = record
        if not record.name in msg_types:
            type = dynamic.generate_dynamic(record.name, record.data.decode())
            msg_types[record.name] = type[record.name]
    if isinstance(record, Channel):
        channels[record.id] = record
    if isinstance(record, Message):
        channel = channels[record.channel_id]
        schema = schemas[channel.schema_id]
        message = msg_types[schema.name]().deserialize(record.data)        
        if schema.name == "sensor_msgs/Imu":
            messages.append(
                {
                    "secs": message.header.stamp.secs,
                    "nsecs": message.header.stamp.nsecs,
                    "x": message.linear_acceleration.x,
                }
            )
print(tabulate.tabulate(messages, headers="keys", tablefmt="pretty"))

+------------+-----------+----------------------+
|    secs    |   nsecs   |          x           |
+------------+-----------+----------------------+
| 1532402927 | 649034000 | -0.41128969192504883 |
| 1532402927 | 659077000 | -0.5330492258071899  |
| 1532402927 | 669063000 |  -0.738662838935852  |
| 1532402927 | 681086000 | -0.5571042895317078  |
| 1532402927 | 689005000 | -0.7012737393379211  |
| 1532402927 | 701017000 | -0.8273831605911255  |
| 1532402927 | 722025000 | -0.7110494375228882  |
| 1532402927 | 726012000 | -0.7359715700149536  |
| 1532402927 | 729008000 | -0.7674471139907837  |
| 1532402927 | 739004000 |  -0.374155730009079  |
| 1532402927 | 749027000 | -0.7169121503829956  |
| 1532402927 | 759016000 | -0.7339585423469543  |
| 1532402927 | 769016000 | -0.6623514890670776  |
| 1532402927 | 781014000 | -0.7557196617126465  |
| 1532402927 | 789011000 | -0.7842211723327637  |
| 1532402927 | 801008000 | -0.49836426973342896 |
| 1532402927 | 823013000 | -0.5673020482063293  |
